In [1]:
import numpy
from numpy import float32, pi, imag, real, mean, minimum, maximum, nanmean, nanstd, absolute
from bokeh.plotting import Session, show, figure, output_server, hplot
from bokeh.models import Range1d
from bokeh.charts import Histogram

from gnss import codes
from gnss.receiver import channels
from gnss.signals import Signal
from gnss.receiver import sources
from gnss.filters import iir_filter
from gnss.acquisition import coarse, fine
from gnss.receiver import outputs
from gnss.tracking import tracking, CN0Algorithm
from gnss.visualization import TrackingPlot, TrackingMultiPlot, CorrelationGridPlot, \
    CorrelationLinePlot, FinePhasePlot, sample_histogram

In [2]:
%reload_ext autoreload
%autoreload 2


We are using data from the USRP front ends. This data contains *signed 4-bit complex* samples.


- L2:
    - 5MHz sampling frequency
    - 1.2276GHz center frequency
- L1:
    - 25MHz sampling
    - 1.57542GHz center frequency
- L5:
    - 25MHz sampling
    - 1.17645GHz center frequency

HIGH RATE DATA

- 0Hz IF
- 100MHz sampling frequency

- high rate center frequency
    - L1: 1.5675GHz
    - L2/L5 band: 1.212GHz

- high rate L1
    - 100MHz sampling frequency
    - 7.92MHz intermediate frequency

- high rate L2/L5
    - L1 7.92MHz intermediate frequency
    - L2 15.6MHz intermediate frequency
    - L5 -35.55MHz intermediate frequency

- high rate GLONASS
    - L1 34.5MHz intermediate frequency
    - L2 34MHz intermediate frequency

- high rate Galileo
    - E1BC 7.92MHz intermediate frequency
    - E5a -35.55MHz intermediate frequency
    - E5b -4.86MHz intermediate frequency

- high rate Beidou
    - B1 -6.402MHz intermediate frequency
    - B2 -4.86MHz intermediate frequency
    

First file: anything in the L1 band

Second file: anything in the low band (L2/L5)

These files are large, so we should choose a reasonable buffer size for their data. 2 seconds worth of data would be:

# DON'T FORGET
Run
    
    > bokeh-server -m --ip <ip>

from the command line.

In [3]:
session = Session(root_url='http://192.168.3.137:5006/', load_from_config=False)
session.login('anon', '1234')
output_server('tracking-l1-ca', session=session)

In [4]:
5e6 * 2, 25e6 * 2

(10000000.0, 50000000.0)

which are 10 mega-samples and 50 mega-samples respectively.

In [5]:
# filepath = '/media/DataStore/x300_l1_test_100MHz.sc4'
# source = sources.FileSource(filepath, file_f_samp=100e6, f_center=1.5675e9, buffer_size=25e6, bit_depth=4, real=False, decimation=2)
# 27min 47sec, fs: 5MHz L1/L2, 25MHz L5  OHz IF

# filepath = '/mnt/gluster/by-drives/Dell/MU-Portable/OUDC3/US_A1_20140808_122714_002747_GPSL1_USRP1.dat'
# source = sources.FileSource(filepath, file_f_samp=5e6, f_center=1.57542e9, buffer_size=5e6, bit_depth=4, real=False, decimation=1)

# filepath = '/mnt/gluster/by-drives/Dell/MU-Portable/x300_compare/csu2a_n200l1_25MHz.sc4'
# filepath = '/home/breitsbw/data/scratch/csu2a_n200l1_25MHz.sc4'
# source = sources.FileSource(filepath, file_f_samp=25e6, f_center=1.57542e9, buffer_size=16e6, bit_depth=4, real=False, decimation=1)

# filepath = '/home/breitsbw/data/scratch/csu2a_n200l1_25MHz.sc4'
# source = sources.FileSource(filepath, file_f_samp=6.25e6, f_center=1.57545e9, buffer_size=24e6, bit_depth=4, real=False)

filepath = '/mnt/gluster/by-drives/WesternDigital/WMC4M0F4FM49/ASI_2013-03-07_L1DISH_2200-2300.dat'
source = sources.FileSource(filepath, file_f_samp=40e6, f_center=1.57542e9, buffer_size=32e6, bit_depth=16, real=False, i_msb=True)

In [6]:
c_acq_plot = CorrelationLinePlot()
f_acq_plot = FinePhasePlot()
show(hplot(c_acq_plot.plot, f_acq_plot.plot))

In [7]:
step = 2e-3
cn0_algo = CN0Algorithm()
f_update = 1. / step
block_duration = 2e-3
block_size = block_duration * source.f_samp - 1
output_buff_size = source.buffer_size / (step * source.f_samp)

In [8]:
i_corr_plot = TrackingPlot('i corr', output_buff_size)
q_corr_plot = TrackingPlot('q corr', output_buff_size)
phase_plot = TrackingPlot('phase error', output_buff_size)
show(hplot(i_corr_plot.plot, q_corr_plot.plot, phase_plot.plot))
buffer_size = round(source.file_duration)
doppler_plot = TrackingPlot('doppler 1Hz', buffer_size)
cn0_plot = TrackingPlot('cn0 1Hz', buffer_size)
show(hplot(doppler_plot.plot, cn0_plot.plot))

In [9]:
source.seek(time=0)
source.load()
print('we have {0} seconds of data starting at {1} seconds'.format(source.buffer_size / source.f_samp, source.buffer_start_time))

we have 0.8 seconds of data starting at 0.0 seconds


In [10]:
svid = 29
signal = Signal.GPSL1CA(svid)

c_acq = coarse.CoarseAcquirerLowMem(source, 10e-3, 2, dopp_min=-5000, dopp_max=5000)
f_acq = fine.FineAcquirer(source, 2e-3, 1e-3, 29)

In [11]:
c_acq.acquire(signal)
chip0 = c_acq.chip
f_dopp0 = c_acq.f_dopp
time = c_acq.time
cn0 = c_acq.cn0
print(chip0, f_dopp0, time, cn0)
c_acq_plot.update(c_acq)

941.7738 -2200.0 0.0 96.6417213093


In [12]:
f_acq.acquire(signal, time, chip0, f_dopp0)
f_dopp = f_acq.f_dopp
time = f_acq.time
print(chip0, f_dopp, time)
f_acq_plot.update(f_acq)

941.7738 -2181.52561393 0.0
0.0 0.056 1.92337722678 -0.424781769254


In [13]:
dll_filter = iir_filter.FirstOrderLowpass(10, f_update)
pll_filter = iir_filter.FirstOrderLowpass(12, f_update)
# pll_filter = iir_filter.SecondOrderLowpass(omega_n=5 * 12, zeta=1.7, fs=f_update)
# pll_filter = iir_filter.SecondOrderLowpassV2(omega_n=5 * 32, zeta=1.7, fs=f_update)
fll_filter = iir_filter.FirstOrderLowpass(1 / pi, f_update);  fll_filter.f(0);  fll_filter.f(0)

delay = .5
correlator = tracking.Correlator(chip_delays=[-2 * delay, -delay, 0., delay, 2 * delay])

In [14]:
time = f_acq.time
theta = last_phase_error = 0.
chip = chip_out = chip0
f_dopp = f_acq.f_dopp
source.seek(time)
source.load()

In [15]:
while True:
    while time < source.max_time - block_duration:
        code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
        earlier, early, prompt, late, later = code_corr
        chip_error = tracking.delay_discriminator(earlier, prompt, later)
        phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
        chip_error = dll_filter.f(chip_error)
        phase_error = pll_filter.f(phase_error)
        dopp_error = (phase_error - last_phase_error) / (step * 2 * pi)
        last_phase_error = phase_error
        dopp_error = fll_filter.f(dopp_error)
        # error correction
        chip -= chip_error
        theta -= phase_error
#         f_dopp += dopp_error
        # carrier-aiding and propagation
        time += step
        f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
        chip += step * f_chip
        f_inter = signal.f_carrier - source.f_center
        theta += step * (f_inter + f_dopp) * 2. * pi
        # outputs
        chip_out = chip_out - chip_error
        i_corr_plot.push_data(time, real(prompt))
        q_corr_plot.push_data(time, imag(prompt))
        phase_plot.push_data(time, phase_error)
#         print(f_dopp)
    data = phase_plot.data.get()
    f_dopp -= mean(data) / (2 * pi) / step
    print(f_dopp)
    i_corr_plot.update()
    q_corr_plot.update()
    phase_plot.update()
    cn0 = cn0_algo.compute(i_corr_plot.data.get(), q_corr_plot.data.get(), step)
    doppler_plot.push_data(time, f_dopp)
    cn0_plot.push_data(time, cn0)
    doppler_plot.update()
    cn0_plot.update()
    source.seek(time)
    source.load()

-2180.92062936
-2180.7940902
-2181.04232009
-2181.24963347
-2181.32313705
-2181.49029325
-2181.83407897


KeyboardInterrupt: 

Something's wrong at 2.5 seconds.

.

.

PARTIAL TRACKING

In [25]:
# time = 0.0
# source.reset()
time = f_acq.time

In [19]:
theta = last_phase_error = 0.
chip = chip_out = chip0
while time < source.max_time - 2 * block_length:
    code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
    early, prompt, late = code_corr
    chip_error = tracking.delay_discriminator(early, prompt, late)
    phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
    chip_error = dll_filter.f(chip_error)
    phase_error = pll_filter.f(phase_error)
    dopp_error = (phase_error - last_phase_error) / (step * 2 * pi)
    last_phase_error = phase_error
    dopp_error = fll_filter.f(dopp_error)
    # error correction
    chip -= chip_error
    theta -= phase_error
    f_dopp -= dopp_error
    # carrier-aiding and propagation
    time += step
    f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
    chip += step * f_chip
    f_inter = signal.f_carrier - source.f_center
    theta += step * (f_inter + f_dopp) * 2. * pi
    # outputs
    chip_out = chip_out - chip_error
    track_plot.push_data(time, real(prompt), imag(prompt), phase_error, f_dopp)

In [20]:
# fig = output.plot(library='matplotlib')
# plt.show()
track_plot.update()

In [3]:
from numpy import arange
from numpy.random import rand
from bokeh.plotting import output_notebook, show
from bokeh.plotting import Plot, GridPlot
from bokeh.models import ColumnDataSource, DataRange1d
from bokeh.models.glyphs import Line

output_notebook()
time = arange(20)
y = rand(len(time))
data = ColumnDataSource(data=dict(time=time, y=y))
x_range = DataRange1d(sources=[data.columns('time')])
y_range = DataRange1d(sources=[data.columns('y')])
plot = Plot(x_range=x_range, y_range=y_range, title='data', plot_width=250, plot_height=250)
# plot = Plot(x_range=None, y_range=None, title='data', plot_width=250, plot_height=250)  # < ahh needs x_range/y_range!
plot.add_glyph(data, Line(x='time', y='y'))
show(plot)